テンプレートの内容を利用し，国旗画像のURLを取得せよ．（ヒント: [MediaWiki API](https://www.mediawiki.org/wiki/API:Main_page/ja)の[imageinfo](https://www.mediawiki.org/wiki/API:Properties/ja#imageinfo_.2F_ii)を呼び出して，ファイル参照をURLに変換すればよい）

In [1]:
from collections import OrderedDict
import json
import re
from urllib import request, parse

import pandas as pd

In [2]:
def extract_by_title(title):
    df_wiki = pd.read_json('jawiki-country.json', lines=True)
    return df_wiki[(df_wiki['title'] == title)]['text'].values[0]

In [3]:
wiki_body = extract_by_title('イギリス')

In [4]:
# rを先頭にするとraw string でエスケープシーケンス無視
# 3重クォートで途中改行無視
# re.VERBOSEオプションを使うことによって、空白とコメント無視
# 非貪欲マッチにすることで、短い文字列を検索
basic = re.findall(r'''
                    ^\{\{基礎情報.*?\n  #検索語句(\はエスケープ処理)、非キャプチャ、非貪欲
                    (.*?)              #任意の文字列
                    \}\}               #検索語句(\はエスケープ処理)
                    $                  #文字列の末尾
                    ''', wiki_body, re.MULTILINE+re.VERBOSE+re.DOTALL)

In [5]:
# rを先頭にするとraw string でエスケープシーケンス無視
# 3重クォートで途中改行無視
# re.VERBOSEオプションを使うことによって、空白とコメント無視
templates = OrderedDict(re.findall(r'''
                          ^\|         # \はエスケープ処理、非キャプチャ
                          (.+?)       # キャプチャ対象(key)、非貪欲
                          \s*         # 空白文字0文字以上
                          =           # 検索語句、非キャプチャ
                          \s*         # 空白文字0文字以上
                          (.+?)       # キャプチャ対象(Value)、非貪欲
                          (?:         # キャプチャ対象外のグループ開始
                            (?=\n\|)  # 改行(\n)+'|'の手前(肯定の先読み)
                          | (?=\n$)   # または、改行(\n)+終端の手前(肯定の先読み)
                          )           # キャプチャ対象外のグループ終了
                         ''', basic[0], re.MULTILINE+re.VERBOSE+re.DOTALL))

In [6]:
# マークアップ除去
def remove_markup(string):
    
    # 強調マークアップの除去 
    # 除去対象：''他との区別''、'''強調'''、'''''斜体と強調'''''
    replaced = re.sub(r'''
                       (\'{2,5})   # 2〜5個の'（マークアップの開始）
                       (.*?)       # 任意の1文字以上（対象の文字列）
                       (\1)        # 1番目のキャプチャと同じ（マークアップの終了）
                       ''', r'\2', string, flags=re.MULTILINE+re.VERBOSE)

    # 内部リンク・ファイルの除去 
    # 除去対象：[[記事名]]、[[記事名|表示文字]]、[[記事名#節名|表示文字]]、[[ファイル:Wi.png|thumb|説明文]]
    replaced = re.sub(r'''
        \[\[             # '[['(マークアップ開始)
        (?:              # キャプチャ対象外のグループ開始
            [^|]*?       # '|'以外の文字0文字以上、非貪欲
            \|           # '|'
        )*?              # グループ終了、このグループが0以上出現、非貪欲(No27との変更点)
        (                # グループ開始、キャプチャ対象
          (?!Category:)  # 否定の先読(含んだ場合は対象外としている)
          ([^|]*?)    # '|'以外が0文字以上、非貪欲(表示対象の文字列)
        )
        \]\]        # ']]'（マークアップ終了）        
        ''', r'\1', replaced, flags=re.MULTILINE+re.VERBOSE)

    # Template:Langの除去 
    # 除去対象：{{lang|言語タグ|文字列}}
    replaced = re.sub(r'''
        \{\{lang    # '{{lang'(マークアップ開始)
        (?:         # キャプチャ対象外のグループ開始
            [^|]*?  # '|'以外の文字が0文字以上、非貪欲
            \|      # '|'
        )*?         # グループ終了、このグループが0以上出現、非貪欲
        ([^|]*?)    # キャプチャ対象、'|'以外が0文字以上、非貪欲(表示対象の文字列)
        \}\}        # '}}'(マークアップ終了)
        ''', r'\1', replaced, flags=re.MULTILINE+re.VERBOSE)
    
    # 外部リンクの除去
    # 除去対象[http(s)://xxxx] 、[http(s)://xxx xxx]
    replaced = re.sub(r'''
        \[https?:// # '[http://'(マークアップ開始)
        (?:           # キャプチャ対象外のグループ開始
            [^\s]*? # 空白以外の文字が0文字以上、非貪欲
            \s      # 空白
        )?          # グループ終了、このグループが0か1出現
        ([^]]*?)    # キャプチャ対象、']'以外が0文字以上、非貪欲（表示対象の文字列）
        \]          # ']'(マークアップの終了)
        ''', r'\1', replaced, flags=re.MULTILINE+re.VERBOSE)

    # HTMLタグの除去
    # 除去対象 <xx> </xx> <xx/>
    replaced = re.sub(r'''
        <           # '<'(マークアップの開始)
        .+?         # 1文字以上、非貪欲
        >           # '>'(マークアップの終了)
        ''', '', replaced, flags=re.MULTILINE+re.VERBOSE)

    return replaced

In [7]:
for i, (key, value) in enumerate(templates.items()):
    replaced = remove_markup(value)
    templates[key] = replaced

In [8]:
print(templates['国旗画像'])

Flag of the United Kingdom.svg


In [9]:
# リクエスト生成
url = 'https://www.mediawiki.org/w/api.php?' \
    + 'action=query' \
    + '&titles=File:' + parse.quote(templates['国旗画像']) \
    + '&format=json' \
    + '&prop=imageinfo' \
    + '&iiprop=url'

In [10]:
# MediaWikiのサービスへリクエスト送信
connection = request.urlopen(request.Request(url))

# jsonとして受信
response = json.loads(connection.read().decode())

In [11]:
print(response['query']['pages']['-1']['imageinfo'][0]['url'])

https://upload.wikimedia.org/wikipedia/commons/a/ae/Flag_of_the_United_Kingdom.svg
